#### Don't Panic 2
https://www.codingame.com/ide/puzzle/don't-panic-episode-2

In [1]:
input_data = '''13 69 79 11 39 8 5 30
6 65
11 4
8 34
8 56
7 17
8 1
2 24
11 13
10 23
6 13
6 34
5 4
1 50
5 46
3 17
10 3
11 42
1 17
1 4
2 23
8 66
2 3
1 24
1 34
8 9
2 58
11 11
11 38
8 23
6 57
0 33 RIGHT
-1 -1 NONE
-1 -1 NONE
0 33 RIGHT
1 33 RIGHT
-1 -1 NONE
0 33 RIGHT
1 33 LEFT
1 32 LEFT
1 31 LEFT
1 30 LEFT
1 29 LEFT
1 28 LEFT
1 27 LEFT
1 26 LEFT
1 25 LEFT
1 24 LEFT
2 24 LEFT
3 24 LEFT
3 23 LEFT
3 22 LEFT
3 21 LEFT
3 20 LEFT
3 19 LEFT
3 18 LEFT
3 17 LEFT
4 17 LEFT
3 18 LEFT
3 17 LEFT
4 17 LEFT
5 17 LEFT
5 16 LEFT
5 15 LEFT
5 14 LEFT
5 13 LEFT
5 12 LEFT
5 11 LEFT
5 10 LEFT
5 9 LEFT
5 8 LEFT
5 7 LEFT
5 6 LEFT
5 5 LEFT
5 4 LEFT
6 4 LEFT
5 5 LEFT
5 4 LEFT
6 4 RIGHT
6 5 RIGHT
6 6 RIGHT
6 7 RIGHT
6 8 RIGHT
6 9 RIGHT
6 10 RIGHT
6 11 RIGHT
6 12 RIGHT
6 13 RIGHT
7 13 RIGHT
7 14 RIGHT
7 15 RIGHT
7 16 RIGHT
7 17 RIGHT
8 17 RIGHT
8 18 RIGHT
8 19 RIGHT
8 20 RIGHT
8 21 RIGHT
8 22 RIGHT
8 23 RIGHT
9 23 RIGHT
8 22 RIGHT
8 23 RIGHT
9 23 RIGHT
10 23 RIGHT
11 23 RIGHT
11 24 RIGHT
11 25 RIGHT
11 26 RIGHT
11 27 RIGHT'''

Refactored version of algo

In [2]:
# Custom input source
from collections import deque

# REMOVE IN CODINGAME
input_stack = deque()
with open('few_clones.txt') as file_object:
    input_stack = deque(file_object.read().split('\n'))
input = lambda: input_stack.popleft()
# REMOVE IN CODINGAME

In [3]:
from __future__ import annotations
import dataclasses
import sys
from collections import deque
from typing import Dict, List, Set, Tuple, Union
from dataclasses import dataclass, field
import datetime
from queue import PriorityQueue

def log(*args):
    print(*args, file=sys.stderr, flush=True)

# nb_floors: number of floors
# width: width of the area
# nb_rounds: maximum number of rounds
# exit_floor: floor on which the exit is found
# exit_pos: position of the exit on its floor
# nb_total_clones: number of generated clones
# nb_additional_elevators: number of additional elevators that you can build
# nb_elevators: number of elevators
nb_floors, width, nb_rounds, exit_floor, exit_pos, nb_total_clones, nb_additional_elevators, nb_elevators = [int(i) for i in input().split()]
nb_floors = exit_floor + 1

# exit floors
exits: Dict[int, Set[int]] = {}
for i in range(nb_floors):
    exits[i] = set()
exits[exit_floor].add(exit_pos)

# elevators
for i in range(nb_elevators):
    # elevator_floor: floor on which this elevator is found
    # elevator_pos: position of the elevator on its floor
    elevator_floor, elevator_pos = [int(j) for j in input().split()]
    exits[elevator_floor].add(elevator_pos)

# how many elevators left to optimize the path
nb_spare_elevators = \
    nb_additional_elevators - sum([not exits[f] for f in range(nb_floors - 1)])

######################################################################################
# Optimize map

# search neighbors
neighbor_el_pos_cache = {}
def neighbor_el_pos(
    floor: int, pos: int
) -> Tuple[Union[int, None], Union[int, None]]:
    loc = floor, pos
    if loc in neighbor_el_pos_cache:
        return neighbor_el_pos_cache[loc]

    to_left, to_right = None, None
    for el_pos in sorted(list(exits[floor])):
        if el_pos > pos:
            to_right = el_pos
            break
        if el_pos < pos:
            to_left = el_pos

    neighbor = to_left, to_right

    neighbor_el_pos_cache[loc] = neighbor
    return neighbor

# allowed limits for each floor
# force elevators for the narrow upper intervals
forced_elevators: Dict[int, int] = {}
allowed_limits: Dict[int, Tuple[int, int]] = {}
l_limit = r_limit = exit_pos # start by allowing any pos, with exit being required
for f in range(nb_floors - 1, -1, -1):
    prev_l, prev_r = l_limit, r_limit

    # look for neighbors of required_pos
    # allow first layer of wrapping elevators (drones can't jump over elevators)
    l_limit, next_after_left = neighbor_el_pos(f, l_limit)
    pref_before_right, r_limit = neighbor_el_pos(f, r_limit)

    # if no elevators in the previous interval
    if f != nb_floors - 1 and nb_spare_elevators > 0 and (
        next_after_left and prev_r and next_after_left > prev_r or
        pref_before_right and prev_l and pref_before_right < prev_l
    ):
        forced_elevators[f] = exit_pos
        nb_spare_elevators -= 1


    if l_limit is None:
        l_limit = 0
    if r_limit is None:
        r_limit = width - 1
        if l_limit == 0: break # stop imposing limits
    
    allowed_limits[f] = l_limit, r_limit

# List of elevators including forced ones
exits_incl_forced = {f: elevators.copy() for f, elevators in exits.items()}
for f in forced_elevators:
    exits_incl_forced[f].add(forced_elevators[f])
    
# log('forced_elevators', forced_elevators)
# log('allowed_limits', allowed_limits)

###################################################################################### 

CLONE_COST = 3
Loc = Tuple[int, int] # floor / pos
LocWParent = Tuple[int, int, bool, Union[Tuple, None]]

@dataclass(order=True)
class KeyPos:
    cost: int = field(compare=False) # rounds to go up through this ELEVATOR
    priority: int
    floor: int = field(compare=False) # floor elevator is on
    pos: int = field(compare=False)
    goes_right: bool = field(compare=False) # clone is going right when reaching this
    parent: None = field(default = None, compare = False)
    spare_elevators: int = field(default = nb_spare_elevators, compare = False)
    clones_remaining: int = field(default = nb_total_clones, compare = False)

# KeyPos builders vvv

def go_to_elevator(crt: KeyPos, next_pos: int):
    nxt = dataclasses.replace(crt)
    nxt.parent = crt

    # Position and orientation
    nxt.floor += 1
    nxt.pos = next_pos
    if nxt.pos != crt.pos:
        nxt.goes_right = (nxt.pos > crt.pos)

    # Cost and priority
    nxt.cost += cost_to_gate(crt, next_pos)
    if nxt.goes_right != crt.goes_right:
        nxt.cost += CLONE_COST
    nxt.priority = nxt.cost + dist_to_exit(crt, nxt.pos)

    # Game state
    if nxt.goes_right != crt.goes_right:
        nxt.clones_remaining -= 1
    
    return nxt

def new_elevator(crt: KeyPos, next_pos: int, using_spare = False):
    nxt = go_to_elevator(crt, next_pos)

    # Cost and priority
    nxt.cost += CLONE_COST
    nxt.priority += CLONE_COST
    
    # Game state
    nxt.clones_remaining -= 1
    if using_spare: nxt.spare_elevators -= 1

    return nxt

# KeyPos builders ^^^

def is_elevator_disallowed(floor: int, pos: int, goes_right: bool):
    return floor + 1 in allowed_limits and (
        pos <= allowed_limits[floor + 1][0] or 
        pos >= allowed_limits[floor + 1][1]
    )

def cost_to_gate(crt: KeyPos, next_pos: int):
    return abs(next_pos - crt.pos) + 1 #+ (nb_floors - crt.floor) + abs(next_pos - exit_pos)

def dist_to_exit(crt: KeyPos, next_pos: int):
    return (nb_floors - crt.floor) + abs(next_pos - exit_pos)

# plan 1: search fastest path; if no elevator ignore it
# (one will be build just above best one)
def bfs_a_star(init_floor: int, init_pos: int, init_right: bool):
    root = KeyPos(0, 0, init_floor - 1, init_pos, True)

    q: PriorityQueue[KeyPos] = PriorityQueue()
    q.put(root)

    visited = set()

    i = 0
    while q:
        crt = q.get()
        i += 1
        if i > 10000: break

        if is_elevator_disallowed(crt.floor, crt.pos, crt.goes_right):
            continue
        if crt.clones_remaining < 0: continue
        # if crt.cost > nb_rounds: continue

        # visit = (crt.floor, crt.pos, crt.goes_right)
        # if visit in visited:
        #     log(visit, crt)
        # visited.add(visit)

    
        if crt.floor == exit_floor and crt.pos == exit_pos:
            elevator_chain: Dict[int, int] = {}
            while crt is not None:
                elevator_chain[crt.floor] = crt.pos
                crt = crt.parent
            return elevator_chain

        next_floor = crt.floor + 1
        if next_floor >= nb_floors: continue

        # Next level options
        next_neighbors = neighbor_el_pos(next_floor, crt.pos)
        if crt.pos in exits[next_floor]:
            # elevator exactly above
            # can only reach neighbor when changing direction
            if crt.goes_right:
                next_neighbors = next_neighbors[0], crt.pos
            else:
                next_neighbors = crt.pos, next_neighbors[1]

        # forced elevator
        if next_floor in forced_elevators:
            if forced_elevators[next_floor] not in next_neighbors:
                continue

            q.put(new_elevator(crt, forced_elevators[next_floor]))
            continue

        # search through neighbors
        elevator_found = False
        for next_pos in next_neighbors:
            if next_pos is None: continue
            
            elevator_found = True
            q.put(go_to_elevator(crt, next_pos))
        
        # Try with a new elevator: forced, empty floor or to optimize
        if not elevator_found: # empty floor
            q.put(new_elevator(crt, crt.pos))
            pass
            
        # for path optimization
        elif crt.spare_elevators > 0:
            # log('crt.spare_elevators', crt.spare_elevators)
            pass
            # optimizaton_positions = set([crt.pos])

            # if crt.floor < nb_floors - 2:
            #     # only do this when 2+ more levels up
            #     one_above = exits_incl_forced[next_floor]
            #     two_above = exits_incl_forced[next_floor + 1]
                
            #     for pos in range(width):
            #         if pos in two_above:
            #             pass
            #         pass

            # q.put(new_elevator(crt, crt.pos, True))

# issue: there is a limit of clones to use for direction switches
# plan 2: A* + add more extra-elevator options

######################################################################################

# game loop
blocked_first = False
optimal_exits = None
first_run = True
while True:
    inputs = input().split()
    clone_floor = int(inputs[0])  # floor of the leading clone
    clone_pos = int(inputs[1])  # position of the leading clone on its floor
    direction = inputs[2]  # direction of the leading clone: LEFT or RIGHT

    # log(clone_floor, clone_pos, direction)

    # No clone yet
    if direction == 'NONE':
        print('WAIT')
        continue

    # search best run
    if first_run:
        optimal_exits = bfs_a_star(clone_floor, clone_pos, direction == 'RIGHT')
        # log('optimal_exits', optimal_exits)
        first_run = False
    if not optimal_exits:
        continue

    # No elevator
    if not exits[clone_floor] or (
        optimal_exits[clone_floor] not in exits[clone_floor] and
        optimal_exits[clone_floor] == clone_pos
    ):
        print('ELEVATOR')
        exits[clone_floor].add(clone_pos)
        continue

    is_dir_correct = False
    exit_dir = \
        'LEFT' if optimal_exits[clone_floor] <= clone_pos else 'RIGHT'
    is_dir_correct = (
        exit_dir == direction or
        clone_pos == optimal_exits[clone_floor]
    )
    
    # Write an action using print
    print('WAIT' if is_dir_correct else 'BLOCK')


KeyboardInterrupt: 